In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import seaborn as sns
import datetime
from collections import OrderedDict
from scipy import stats
import corner
from sklearn.preprocessing import MinMaxScaler

from mlxtend.regressor import StackingRegressor
from mlxtend.data import boston_housing_data
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import numpy as np
import warnings

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

from matplotlib.pyplot import figure

plt.style.use('seaborn')
%matplotlib inline

# Since I have already preprocessed the data and used `sklearn train_test_split` we have three separate files needed to be imported

In [2]:
X_train = pd.read_csv('X.csv')
Y_train = pd.read_csv('y.csv')

X_test = pd.read_csv('test_cleaned.csv')

### Identifying Multicollinearity is very important without doing so its likely that we would be overfitting. However, I feel like this is only towards the case with much simpler models such as linear regression and the fact that we run some sort of boosted trees won't really affect much. But it is good to identify some of these predictor variables that have high VIFs (Variance Inflation Factors)

In [10]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
vif.round(1).head()

,VIF Factor,features
0,3.5,LotFrontage
1,4.4,LotArea
2,1.5,Street
3,6.6,Alley
4,1.6,LotShape


# Its likely that we have to remove some variables that have VIF values over 5 so alley is one variable we may need to consider

# Root Mean Squared Error (RMSE) function to evaluate results
* RMSE represents the sample standard deviation of the differences between predicted values and observed values (called residuals).
* Since the errors are squared before they are averaged, RMSE gives a relatively high weight to large errors (Penalizes it harder than MAE)

* This means the RMSE should be more useful when large errors are particularly undesirable.

* From an interpretation standpoint, MAE has the edge. RMSE does not describe average error alone and has other implications that are more difficult to tease out and understand.

* However, an advantage of RMSE over MAE is that RMSE avoids the use of taking the absolute value, which is undesirable in terms of calculations and applications

* In addition, What are good rmse values?
    * This really depends on the depedent Y variables 
    * when train rmsse is similar to test rmse

# Lets define some rmse functions
* Note most sklearn models have some sort of evaluation metric similar to rmse such as negative mean square error. All we have to do is tweek it slightly and we can get the rmse.

In [7]:
def rmse(predictions, targets): 
    return np.sqrt(((predictions - targets) ** 2).mean())

def rmse_cv(model, x_train, y_train):
        rmse = np.sqrt(-cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=5))
        return rmse.mean()

In [20]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

seed = 33

lasso = Lasso()
linear_reg = LinearRegression()
ridge = Ridge(random_state=1)
svr_linear = SVR(kernel='linear')
rf = RandomForestRegressor(n_estimators=5, random_state=seed)
ridge = Ridge(random_state=seed)
ElasticNet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0003, l1_ratio=.9, random_state=3,max_iter = 10000))
GradBoost = GradientBoostingRegressor(n_estimators=3300, learning_rate=0.03,
                                   max_depth=6, max_features='sqrt',
                                   min_samples_leaf=13, min_samples_split=9, 
                                   loss='huber', random_state =5)

xgb = xgb.XGBRegressor(objective= 'reg:squarederror', colsample_bytree=0.4, gamma=0.05, 
                             learning_rate=0.05, max_depth=5, 
                             min_child_weight=2, n_estimators=3000,
                             reg_alpha=0.5, reg_lambda=0.9,
                             subsample=0.5, silent=1,
                             random_state =7, nthread = -1)

Kernel_Ridge = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)


stack = StackingCVRegressor(regressors=(linear_reg, lasso, ridge, svr_linear, rf, ElasticNet, GradBoost, xgb, Kernel_Ridge),
                            meta_regressor=xgb,
                            random_state=seed)

print('5-fold cross validation scores:\n')
for clf, label in zip([linear_reg, lasso, ridge, svr_linear, rf, ElasticNet, GradBoost, xgb, Kernel_Ridge, stack], ['Linear Regression','Lasso', 'Ridge','SVR', 'Random Forest', 'Elastic Net','Gradient Boosting','XGBoost Regressor','Kernel Ridge',  'StackingCVRegressor']):
    scores_R2 = cross_val_score(clf, X_train.as_matrix(), Y_train, cv=5)
    scores_rmse = np.sqrt(-cross_val_score(clf, X_train.as_matrix(), Y_train, cv=5, scoring='neg_mean_squared_error'))
    print("R^2 Score: %0.5f (+/- %0.5f) [%s]" % (scores_R2.mean(), scores_R2.std(), label))
    print("RMSE Score: %0.5f (+/- %0.5f) [%s] \n" % (scores_rmse.mean(), scores_rmse.std(), label))

5-fold cross validation scores:

R^2 Score: 0.90126 (+/- 0.01012) [Linear Regression]
RMSE Score: 0.12531 (+/- 0.01015) [Linear Regression] 

R^2 Score: -0.00335 (+/- 0.00490) [Lasso]
RMSE Score: 0.39947 (+/- 0.01584) [Lasso] 

R^2 Score: 0.91125 (+/- 0.00834) [Ridge]
RMSE Score: 0.11876 (+/- 0.00844) [Ridge] 

R^2 Score: 0.89906 (+/- 0.00922) [SVR]
RMSE Score: 0.12669 (+/- 0.00929) [SVR] 

R^2 Score: 0.84365 (+/- 0.01345) [Random Forest]
RMSE Score: 0.15730 (+/- 0.00285) [Random Forest] 

R^2 Score: 0.92057 (+/- 0.00651) [Elastic Net]
RMSE Score: 0.11232 (+/- 0.00673) [Elastic Net] 

R^2 Score: 0.91336 (+/- 0.00523) [Gradient Boosting]
RMSE Score: 0.11744 (+/- 0.00775) [Gradient Boosting] 

R^2 Score: 0.91513 (+/- 0.00778) [XGBoost Regressor]
RMSE Score: 0.11609 (+/- 0.00737) [XGBoost Regressor] 

R^2 Score: 0.91701 (+/- 0.00453) [Kernel Ridge]
RMSE Score: 0.11485 (+/- 0.00561) [Kernel Ridge] 

R^2 Score: 0.91902 (+/- 0.00692) [StackingCVRegressor]
RMSE Score: 0.11343 (+/- 0.00718) [S

## In a perfect world where computation power is not an issue then we can tune every parameter in all the base learners mentioned above. However, that is not the case and I'll just tune a few to show that it works and may we can get a slighlty better prediction?  

In [24]:
stack = stack.fit(X_train.as_matrix(), Y_train)

In [26]:
predictions = stack.predict(X_test.as_matrix())

In [27]:
l = []
for i in predictions:
    l.append(np.exp(i)-1)

In [28]:
predictions = l
predictions  =pd.DataFrame(predictions)

In [29]:
predictions.to_csv('newest_predictions.csv')

In [30]:
predictions.head()

,0
0,117747.328125
1,165011.546875
2,184652.421875
3,193491.421875
4,182182.046875
